In [1]:
import numpy as np
from scipy.spatial import distance

In [79]:
class DBSCAN:
    
    # DBSCAN implementation following https://cse.buffalo.edu/~jing/cse601/fa13/materials/clustering_density.pdf.
    
    # DBSCAN takes two parameters:
    #    eps = threshold distance
    #    min_pts =  number of points required to be within eps
    
    # DBSCAN.fit(x) is passed data in the form of a DxN matrix where D=dimensionality and N=number of datapoints
    
    def __init__(self,eps,min_pts):
        self.eps = eps
        self.min_pts = min_pts
    
        

    def fit(self,x):        
        # Create a cluster vector which holds the clustering assignments
        # An unvisited point has a cluster assignment of -2
        # Noise assignment = -1
        # Cluster assignments >=0

        self.cluster = np.full((x.shape[0],1), -2, dtype=int) 

        
        # Initial cluster = 0  (-1 is immediately increased below)
        cluster_ID = -1

        
        # Iterate though the data
        for p in range(x.shape[0]):
            # If the point is unvisited examine point 
            if self.cluster[p]==-2: 
                
                # Calculate points in the neighbourhood given the threshold value (eps)
                neighbour_pts = self.query_region(x,x[p])
                
                # If there are < min_pts in the neighbourhood this point is classified as noise (-1)
                if len(neighbour_pts)<self.min_pts:
                    self.cluster[p] = -1 
                    
                else:
                    # Increment clustering counter (new cluster)
                    cluster_ID += 1 
                    # Assign point p to new cluster
                    self.cluster[p] = cluster_ID
                    # Use point p as the root node for the clustering to start from by expanding the cluster
                    self.expand_cluster(x,x[p],p,neighbour_pts,cluster_ID)
        
        # Reshape cluster array into readable 1D array
        self.cluster = np.reshape(self.cluster,(len(self.cluster),))
    
    
    # Calculates the distance of x wrt y. Currently using euclidean distance.
    def calc_dist(self,x,y):
        return distance.minkowski(x,y,p=2)

    
    # Calculate points within the threshold radius specified by parameter: eps
    def query_region(self,x,point):
        
        neighbours = np.array([])
        for i in range(x.shape[0]):
            
            # Points within threshold are in neighbourhood region
            if self.calc_dist(point,x[i])<self.eps:
                neighbours = np.append(neighbours,int(i))

        return neighbours
        
        
    # Expand the cluster from root node and assign valid points to cluster_ID    
    def expand_cluster(self,x,point,index,neighbour_pts,cluster_ID):
        
        # Method is passed :
        #                   point = root node, float
        #                   index = index in array of the root point, int
        #                   neighbour_pts = array of neighbourhood for root node, []
        #                   cluster_ID = ID for cluster assignment, int
        
        
        
        # A while loop iterates over neighbour_pts as the length of neighbour_pts may change
        p_ = 0
        while p_ < len(neighbour_pts):
            
            # Find 
            index = neighbour_pts[p_].astype(int)
            
            # If new point in question is not visited (has not been assigned a cluster)
            if self.cluster[index] == -2:
                
                # Find neighbourhood of points in original neighbourhood
                neighbour_pts_ = self.query_region(x,x[index])
                
                
                # Check whether neighbourhood size is valid given parameters (min_pts)
                if len(neighbour_pts_) >= self.min_pts:
                    neighbour_pts = np.concatenate((neighbour_pts,neighbour_pts_))
            
            # If point is not assigned a cluster
            if self.cluster[index]==-2: 
                self.cluster[index]=cluster_ID
            p_+=1
            
# DBSCAN.cluster holds the assigned clusters for each point given parameter settings

In [84]:
x = np.array([[1, 2], [2, 2], [2, 3],[8, 7], [8, 8], [25, 80]])
model = DBSCAN(eps = 2.5, min_pts = 2)
model.fit(x)
print("Cluster assignments:",model.cluster)

Cluster assignments: [ 0  0  0  1  1 -1]
